In [1]:
cd /home/ahmed/development/fastpi_course

/home/ahmed/development/fastpi_course


In [2]:
from app import db
from cassandra.cqlengine.management import sync_table
from app.users.models import User

In [3]:
db.get_session()
sync_table(User)

In [4]:
post_data = {
    'email': 'testpydantic@gmail.com',
    'password': 'abc123'
}

In [8]:
from pydantic import BaseModel, EmailStr

In [6]:
class UserLoginSchema(BaseModel):
    email: str
    password: int

In [7]:
UserLoginSchema(**post_data) # passed as kwargs email='test...', password='abc123'

ValidationError: 1 validation error for UserLoginSchema
password
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='abc123', input_type=str]
    For further information visit https://errors.pydantic.dev/2.4/v/int_parsing

In [10]:
class UserLoginSchema(BaseModel):
    email: EmailStr
    password: str
    

In [11]:
UserLoginSchema(**post_data) # passed as kwargs email='test...', password='abc123'

UserLoginSchema(email='testpydantic@gmail.com', password='abc123')

In [12]:
UserLoginSchema(email='wrongemail', password='123') # passed as kwargs email='test...', password='abc123'


ValidationError: 1 validation error for UserLoginSchema
email
  value is not a valid email address: The email address is not valid. It must have exactly one @-sign. [type=value_error, input_value='wrongemail', input_type=str]

In [13]:
from pydantic import BaseModel, EmailStr, SecretStr

In [14]:
UserLoginSchema(email='wrongemail@gmail.com', password='123') # passed as kwargs email='test...', password='abc123'


UserLoginSchema(email='wrongemail@gmail.com', password='123')

In [15]:
class UserLoginSchema(BaseModel):
    email: EmailStr
    password: SecretStr
    

In [16]:
UserLoginSchema(email='wrongemail@gmail.com', password='123')

UserLoginSchema(email='wrongemail@gmail.com', password=SecretStr('**********'))

In [17]:
# password='123')  vs password=SecretStr('**********'))


In [22]:
from pydantic import BaseModel, EmailStr, SecretStr, field_validator

In [25]:
class UserSignupSchema(BaseModel):
    email: EmailStr
    password: SecretStr
    password_confirmation: SecretStr
    @validator('password_confirmation')
    def password_match(cls, v, values, **kwargs):
        password = values.get('password')
        password_confirmation = v
        if password != password_confirmation:
            raise ValueError("passwords don't match")
        return v
        


/tmp/ipykernel_516519/3186061029.py:5: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  @validator('password_confirmation')


In [24]:
UserSignupSchema(email='test@gmail.com', password='123', password_confirmation='11')

AttributeError: 'pydantic_core._pydantic_core.ValidationInfo' object has no attribute 'get'

In [26]:
class UserSignupSchema(BaseModel):
    email: EmailStr
    password: SecretStr
    password_confirmation: SecretStr
    @field_validator('password_confirmation')
    def password_match(cls, values, **kwargs):
        password = values.get('password')
        password_confirmation = values.get("password_confirmation")
        if password != password_confirmation:
            raise ValueError("passwords don't match")
        return v

In [27]:
UserSignupSchema(email='test@gmail.com', password='123', password_confirmation='11')


AttributeError: 'SecretStr' object has no attribute 'get'

In [97]:
class UserSignupSchema(BaseModel):
    email: EmailStr
    password: SecretStr
    password_confirmation: SecretStr

    @field_validator("password_confirmation")
    def passwords_match(cls,value: SecretStr, info: ValidationInfo):
        # print(type(cls), cls) # class 'pydantic._internal._model_construction.ModelMetaclass
        password = info.data.get('password').get_secret_value()
        password_confirmation = value.get_secret_value()
        if password != password_confirmation:
            raise ValueError("Passwords don't match")


In [98]:
UserSignupSchema(email='test@gmail.com', password='123', password_confirmation='11')

ValidationError: 1 validation error for UserSignupSchema
password_confirmation
  Value error, Passwords don't match [type=value_error, input_value='11', input_type=str]
    For further information visit https://errors.pydantic.dev/2.4/v/value_error

In [93]:
from pydantic import BaseModel, ValidationInfo, field_validator

In [100]:
UserSignupSchema(email='test@gmail.com', password='123', password_confirmation='123')

UserSignupSchema(email='test@gmail.com', password=SecretStr('**********'), password_confirmation=None)

In [103]:
class UserSignupSchema(BaseModel):
    email: EmailStr
    password: SecretStr
    password_confirmation: SecretStr

    @field_validator("password_confirmation")
    def passwords_match(cls,value: SecretStr, info: ValidationInfo):
        # print(type(cls), cls) # class 'pydantic._internal._model_construction.ModelMetaclass
        password = info.data.get('password').get_secret_value()
        password_confirmation = value.get_secret_value()
        if password != password_confirmation:
            raise ValueError("Passwords don't match")

        return password
    
    @field_validator("email")
    def email_exists(cls,value: EmailStr, info: ValidationInfo):
        q = User.objects.filter(email=value)
        print("info ",info)
        print('q= ', q)
        if q.count() != 0:
            raise ValueError('Email is not available')

        return value



In [110]:
data = UserSignupSchema(email='test1@test.com', password='123', password_confirmation='123')

info  ValidationInfo(config={'title': 'UserSignupSchema'}, context=None, data={}, field_name='email')
q=  SELECT "user_id", "password" FROM video_membership_app.user WHERE "email" = %(0)s LIMIT 10000


ValidationError: 1 validation error for UserSignupSchema
email
  Value error, Email is not available [type=value_error, input_value='test1@test.com', input_type=str]
    For further information visit https://errors.pydantic.dev/2.4/v/value_error

In [116]:
#to work with non-indexed PK in datastax and cassaandra use it in this format:
User.objects(user_id='1c82e92e-709d-11ee-b051-e91f02349b61').allow_filtering()